In [1]:
%cd ../../

/home/PJWSTK/s14028/Engine


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v2 import CNN_v2
from CNN.EnsembleCNN import EnsembleCNN

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cop = np.linspace(0, 1, 11)[1:-1]

In [5]:
cnn_v2 = CNN_v2((480, 640, 4), split_into_parts=20)

images = cnn_v2._prepare_images(image_train)
anwsers = cnn_v2._prepare_anwsers(person_coo_train)

In [ ]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [ ]:
anwsers[val_indices[zeros_count:]] = 0

In [ ]:
val_indices = -(images.shape[0] - val_indices)

In [ ]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [ ]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [ ]:
def ensemble_teach(count, x, y, val):
  models = []
  
  for i in range(count):
    print(f"Begin to train {i}-th model.")
    
    model = CNN_v2((480, 640, 4), split_into_parts=20)
    model.def_model()
    
    model.model = keras.utils.multi_gpu_model(model.model, gpus=5, cpu_merge=False)
    model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    model.model.optimizer.lr.assign(1e-3)
    model.model.fit(x, y, batch_size=20000, epochs=30, validation_data=val)
    model.model.optimizer.lr.assign(1e-4)
    model.model.fit(x, y, batch_size=20000, epochs=20, validation_data=val)
    model.model.optimizer.lr.assign(5e-5)
    model.model.fit(x, y, batch_size=20000, epochs=80, validation_data=val)
    
    model.model.save_weights(f"CNN/CNN_v2/weights/ensemble_model_{i}")
    print(f"Model {i}-th finished training", end="\n\n")
    
    models.append(model)
    
  return models

In [6]:
def load_ensemble(count):
  models = []
  
  for i in range(count):
    model = CNN_v2((480, 640, 4), split_into_parts=20)
    model.def_model()
    
    model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
    model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    model.model.load_weights(f"CNN/CNN_v2/weights/ensemble_model_{i}")
    
    models.append(model)

  return models

In [ ]:
models = ensemble_teach(10,
               images[:-validation_length],
               anwsers[:-validation_length],
               (images[-validation_length:], anwsers[-validation_length:]))

In [7]:
models = load_ensemble(10)

In [8]:
model = EnsembleCNN(models, (480, 640, 4), split_into_parts=20)

In [9]:
result = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

In [10]:
diff = best_cop_diff(result, count_test)

print(f"Model EnsembleCNN behaved:")

print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
print(diff[2])

print(f"With MSE {mse(diff[2])}")
print(f"With MAE {mae(diff[2])}", end="\n\n")

Model EnsembleCNN behaved:
For cut-of-point 0.5 had accuracy 79.3817367075983:
[20  8  5  2  9  8  8  3  6  2 -1  4 -1  8  0  5 11  4  9  7  4  8  2  1
 -3  0  3  1  6  5  3  1  3  6 11  2 14  3  6  7 15  9 12 12 11 12 11  7
 12 11 10  2  4  3  7  6 11 15 12  6 18  4  6  6  2  7  2  5 -9  1  2  5
  7  3 -1  7  2  1  1  3 -4  6  3  5 -3  8  5  2  8  8  9  6 12 -2  7 -4
  2  5 -1  4 24 11 10  9 10  2  9 14 10 15  2  1  9 10  9 14  7  4  1  3
  0  7 10  8  9  6 12  7  1  8 11 13 10 10 10  8  8  4 13 14  8  8 12 16
 16 11  9  4  7  3  0  1 10  2  0  4  9 12  8  2  3  0 -2  1  6  3  8  1
  0  3 10  8  9  7  0  2 10  1 13  9  1  3  0  2  5  7 -6 17  2  0  3 11
  7 -2  2 11  5  6  5  2  2  0  7  7  8  6  3  3 -1  0  1  3  7 -3 -5  0
  5 13  9 11 11 -1  1 14  7 10  4  1  7 14 13  9 15 12  0  0 -1  0  5  5
  5 16  5  7 16 12  1 13  8  9 -1  9  7  7  7  5  0  6  3 10  3 10  9  1
  1  6  3  7  2 -1 -1  4  1  7  6  4 14 16  3 13  4  4 14  2  8  6  5 13
  7 10  9 10  1  7  2  5 11  2  2 10]
With MS

In [11]:
new_models = [models[4], models[5], models[9]]

In [12]:
new_model = EnsembleCNN(new_models, (480, 640, 4), split_into_parts=20)

In [13]:
result = test_model(model=new_model, images=image_test, counts=count_test, cut_of_points=cop)

In [14]:
diff = best_cop_diff(result, count_test)

print(f"Model EnsembleCNN behaved:")

print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
print(diff[2])

print(f"With MSE {mse(diff[2])}")
print(f"With MAE {mae(diff[2])}", end="\n\n")

Model EnsembleCNN behaved:
For cut-of-point 0.7000000000000001 had accuracy 76.13866175732656:
[ -8 -17 -15 -15  -7  -4  -3 -11  -5  -8  -8   0  -5  -1  -1  -6  -2  -6
  -5  -6  -4  -4  -4  -9  -9  -7  -7  -9  -2  -5  -5  -8  -4  -5   1  -6
   4  -5 -10  -5  -1  -3   1  -2   1  -4   0  -8  -9  -1  -1  -8   0  -6
  -3   0  -8   2  -4 -10 -14 -18 -10 -12 -12 -11 -14 -13 -22  -9  -8 -12
  -6 -10  -9  -3 -13  -8 -11  -3 -11  -7 -10  -4  -7  -1  -8  -8  -3  -5
  -1  -6  -3 -21 -11 -23 -22  -6 -15 -14  -1  -6  -4  -2  -5  -6  -3  -3
 -11  -5  -8 -12  -9 -12  -6 -11  -9 -12  -7  -6  -9  -8  -7  -9  -7  -5
  -6  -6  -9  -4  -9  -2 -10 -12  -9 -13  -6 -10  -6  -8  -5  -5 -12  -6
  -8 -13  -9  -4 -12  -9  -6  -7  -5 -11 -13  -8  -9  -2  -3 -10 -11 -11
 -10 -11  -5 -13  -3  -9 -11  -9  -3  -3  -3  -5  -3 -15  -2  -7   2  -3
  -6  -8 -10  -9  -7  -5 -16  -7  -6  -3 -10   1  -7 -14 -11   3  -6  -3
  -4  -9  -5  -5 -10  -5  -2   2  -9  -4 -13  -9 -12  -7 -14 -17 -15 -10
 -12   1  -7  -9   1 -12  -7 

In [15]:
cnn_model = CNN_v2((480, 640, 4), split_into_parts=20)
cnn_model.def_model()

cnn_model.model = keras.utils.multi_gpu_model(cnn_model.model, gpus=2, cpu_merge=False)
cnn_model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cnn_model.model.load_weights("CNN/CNN_v2/weights/cnn_v2_5e_5_80")

In [22]:
new_models.append(cnn_model)

In [23]:
new_model = EnsembleCNN(new_models, (480, 640, 4), split_into_parts=20)

In [24]:
result = test_model(model=new_model, images=image_test, counts=count_test, cut_of_points=cop)

In [25]:
diff = best_cop_diff(result, count_test)

print(f"Model EnsembleCNN behaved:")

print(f"For cut-of-point {diff[0]} had accuracy {diff[1]}:")
print(diff[2])

print(f"With MSE {mse(diff[2])}")
print(f"With MAE {mae(diff[2])}", end="\n\n")

Model EnsembleCNN behaved:
For cut-of-point 0.6000000000000001 had accuracy 86.63579936949266:
[ 13   4   2   0   4   4   5   4   8   2  -1   5   2   3   8   1   7   2
   1   1   4   9   2   3  -4   1   1   2   4   0   3  -1   2   4  13   2
  12   3   2   0   8   5   7   9   6   8   7   5   3   9   5  -1   4  -1
   4   9   7  12   5   5   6   6   4   3   1   1   2   0  -7   3   0   1
   5  -6  -2   1  -1   1  -1   5  -3   8   3   0  -2   6   3   5  11   0
   4   1   6  -6   8 -10   1  -2   0   1  13   5   3   2   9  -3   5   7
   4   9   2  -2   1  -1  -3   4  -3   3   0  -1   2   2   7   0   2   4
   7   3  -1   5   4  13   5   0   9   1   9   0   0   1   5   4  -2  16
  11  -2   5   7   2  -3  -3   1   1   1  -4   4   2   7   8   1   1  -3
   3  -1   1  -3   6  -4  -2   2   6   3   3   6  -1   4   3   2  10   3
  -1   5   0  -1   1   4  -9  11   1   1   0   9   6   2   3  13   0   6
   2   4   3   3   4   2   2   4  -2   4  -6  -2   0   2  -6  -7  -6  -4
  -1   6   3   8  10  -2   8 